# Exercise: Exploratory Data Analysis with Pandas
## Instructions
1. Data Collection
    - Search and collect data on the following: (1993 - 2023)
        - Israel interest tares
        - US interest rates
        - USD to ILS exchange rates 
    - For each dataset, create a pandas DataFrame with the following columns:
        - Year
        - Month
        - Current_Rate (Interest rate or exchange rate for that month)
        - Change_From_Last_Month (Difference in Current_Rate from the previous month)
        - Optional: You may include additional columns if you find them useful (e.g., Change_Percentage, Date).



In [1]:
import pandas as pd

In [2]:
df_IL_rates = pd.read_json('DateFiles/israeli_rates.json')
df_IL_rates['time'] = pd.to_datetime(df_IL_rates['time'])
df_IL_rates['Year'] = df_IL_rates['time'].dt.year
df_IL_rates['Month'] = df_IL_rates['time'].dt.month
df_IL_rates.rename(columns={'percent': 'Current_Rate'}, inplace=True)
df_IL_rates['Change_From_Last_Month'] = df_IL_rates['Current_Rate'].diff()
df_IL_rates = df_IL_rates[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]

In [3]:
df_US_rates = pd.read_csv('DateFiles/USD_ILS Historical Data.csv')
df_US_rates['Date'] = pd.to_datetime(df_US_rates['Date'])
df_US_rates['Year'] = df_US_rates['Date'].dt.year
df_US_rates['Month'] = df_US_rates['Date'].dt.month
df_US_rates.rename(columns={'Price': 'Current_Rate'}, inplace=True)
df_US_rates['Change_From_Last_Month'] = df_US_rates['Current_Rate'].diff()
df_US_rates = df_US_rates[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]

In [4]:
df_ex_DTS = pd.read_csv('DateFiles/FEDFUNDS.csv')
df_ex_DTS['DATE'] = pd.to_datetime(df_ex_DTS.DATE)
df_ex_DTS['Year'] = df_ex_DTS['DATE'].dt.year
df_ex_DTS['Month'] = df_ex_DTS['DATE'].dt.month
df_ex_DTS.rename(columns={'FEDFUNDS': 'Current_Rate'}, inplace=True)
df_ex_DTS['Change_From_Last_Month'] = df_ex_DTS['Current_Rate'].diff()
df_ex_DTS = df_ex_DTS[['Year', 'Month', 'Current_Rate', 'Change_From_Last_Month']]

## Part A: Single DataFrame Analysis 
1. Display the first 5 rows of the US interest rates DataFrame.

2. Calculate the average Israeli interest rate over the period covered by your data.

3. Find the month and year when the USD to ILS exchange rate had the highest increase from the previous month.

4. Determine the total number of months where the US interest rate decreased compared to the previous month.

5. Add a new column to the Israeli interest rates DataFrame called Change_Percentage, representing the percentage change from the previous month.



In [6]:
# A - 1
df_US_rates.head()

,Year,Month,Current_Rate,Change_From_Last_Month
0,2023,12,3.6008,NaN
1,2023,11,3.7231,0.1223
2,2023,10,4.0369,0.3138
3,2023,9,3.8080,-0.2289
4,2023,8,3.8009,-0.0071


In [7]:
# A - 2
average_IL_rates = df_IL_rates.Current_Rate.mean()
print(average_IL_rates)

4.7612215909090905


In [8]:
# A - 3
max_ex_DTS_index = df_ex_DTS['Change_From_Last_Month'].idxmax()
year = df_ex_DTS.loc[max_ex_DTS_index, 'Year']
month = df_ex_DTS.loc[max_ex_DTS_index, 'Month']
change = df_ex_DTS.loc[max_ex_DTS_index, 'Change_From_Last_Month']
print(f'date: {month}-{year}\nchange: {change}')


date: 11-2022
change: 0.6999999999999997


In [9]:
# A - 4
sum_months_US_rates = df_US_rates[df_US_rates['Change_From_Last_Month'] < 0].shape[0]
print(sum_months_US_rates)

179


In [10]:
# A - 5
df_IL_rates['Change_Percentage'] = df_IL_rates['Current_Rate'].pct_change()
df_IL_rates.head()

,Year,Month,Current_Rate,Change_From_Last_Month,Change_Percentage
0,1994,9,14.0,NaN,NaN
1,1994,10,15.5,1.5,0.107143
2,1994,11,15.5,0.0,0.000000
3,1994,12,17.0,1.5,0.096774
4,1995,1,17.0,0.0,0.000000


## Part B: Merging DataFrames
1. Merge the US and Israeli interest rates DataFrames on Year and Month.

2. In the merged DataFrame, create a new column Rate_Difference that shows the difference between the Israeli and US interest rates for each month.

3. Identify all months when the Israeli interest rate was at least 1% higher than the US interest rate - IH (Israeli Higher).
Identify all months when the US interest rate was at least 1% higher than the Israeli interest rate - UH (US Higher).
Identify all months when the interest rates were within 1% of each other - N (Near)
make a new column called Rate_Comparison with the following values: IH, UH, or N.

4. Merge the exchange rate DataFrame with the merged interest rates DataFrame.

5. After merging all three DataFrames, find the correlation between the Rate_Difference and the Current_Rate of the exchange rate.
hint: find the connection between the difference in interest rates and the exchange rates, use any method you think is appropriate.
provide an explanation of the results (in Hebrew) - at least 2 rows of text + a proof of the connection(some output) you found.


In [11]:
# B - 1
merged_df = pd.merge(df_US_rates, df_IL_rates, on=['Year', 'Month'], suffixes=('_US', '_IL'))
merged_df.head()

,Year,Month,Current_Rate_US,Change_From_Last_Month_US,Current_Rate_IL,Change_From_Last_Month_IL,Change_Percentage
0,2023,12,3.6008,NaN,4.75,0.0,0.0
1,2023,11,3.7231,0.1223,4.75,0.0,0.0
2,2023,10,4.0369,0.3138,4.75,0.0,0.0
3,2023,9,3.8080,-0.2289,4.75,0.0,0.0
4,2023,8,3.8009,-0.0071,4.75,0.0,0.0


In [ ]:
# B - 2
merged_df['Rate_Difference'] = merged_df['Current_Rate_IL'] - merged_df['Current_Rate_US']
merged_df.head()

In [ ]:
# B - 3
merged_df['Rate_Comparison'] = merged_df['Rate_Difference'].apply(
    lambda x: 'IH' if x >= 1 else ('UH' if x <= -1 else 'N')
)
merged_df.head()

In [ ]:
# B - 4
final_df = pd.merge(merged_df, df_ex_DTS, on=['Year', 'Month'], how='left', suffixes=('', '_Exchange'))
final_df.head()


In [ ]:
# B - 5
correlation = final_df['Rate_Difference'].corr(final_df['Current_Rate'])

print(correlation)


## Part C: Analyzing Relationships Between DataFrames
1. Analyze whether increases in US interest rates are associated with increases or decreases in the USD to ILS exchange rate in the same month.

2. Determine if changes in the Israeli interest rate have a stronger impact on the exchange rate than changes in the US interest rate.

3. Find any time lags in the effect of interest rate changes on the exchange rate. For example, does a change in US interest rates affect the exchange rate in the following month?
(כלומר- האם כשיש שינוי בריבית כלשהי - האם מייד יש שינוי בשער ההמרה? או שיש דיליי של חודש/חודשים עד שמרגישים את ההשפעה - אם בכלל?)

4. Calculate the rolling past 3-month average of the exchange rate for each data time, and add it as a new column.

5. Identify any periods where both US and Israeli interest rates were decreasing, and analyze how the exchange rate behaved during those periods.



In [ ]:
# Bonus 

# Bonus
choose 3 global / local events or periods that you think could have affected the exchange rate or interest rates
and analyze the effect of these events on the data you collected.
show proof of your analysis that supports your claim.

(כלומר- תבחנו אירועים שקרו בתקופה שאתם עובדים עליה ובדקו איך הם השפיעו על הנתונים שאתם עובדים איתם)
למשל: בחירות בארה״ב, בחירות בישראל, רוסיה אוקראינה, מלחמות בישראל וכו׳.

In [ ]:
# Your code here